In [1]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator

%aimport GraphAPI

## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [17]:
gc = GraphCreator("Sonata form")
print("Number of Links to Search:", len(gc.next_links))
print(gc.intro_nodes)

Number of Links to Search: 311
['Musical form', 'Classical music era', 'Movement (music)', 'Tonality', 'Exposition (music)', 'Musical development', 'Recapitulation (music)', 'Introduction (music)', 'Coda (music)', 'Sonata', 'Symphony', 'Concerto', 'String quartet', 'Musical analysis']


2. We query all the nodes linked to from the entry point (expand our network one level for each node).

In [18]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [19]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [20]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Ludwig van Beethoven,Wolfgang Amadeus Mozart,21
1,Wolfgang Amadeus Mozart,Ludwig van Beethoven,21
2,Carl Maria von Weber,Ludwig van Beethoven,17
3,Wolfgang Amadeus Mozart,Joseph Haydn,17
4,Ludwig van Beethoven,Carl Maria von Weber,17


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [21]:
features_df = gc.get_features_df(rank=False)
gc.features_df

,node,degree,category_matches_with_source,in_edges,out_edges,centrality,dispersion,page_rank,adjusted_reciprocity,shortest_path_length_from_source
0,Baroque,8202,1,7672.0,530.0,7.233459e-03,1.400000,0.040093,406.0,2.0
1,Orchestra,4930,0,4581.0,349.0,3.068620e-02,0.730769,0.016732,272.0,1.0
2,Wolfgang Amadeus Mozart,4921,0,4555.0,366.0,3.179824e-01,2.580645,0.016299,418.0,1.0
3,Ludwig van Beethoven,4874,1,4194.0,680.0,3.901198e-01,1.043478,0.014720,904.0,2.0
4,Tempo,3717,0,3476.0,241.0,2.347077e-02,1.000000,0.011467,262.0,1.0
5,Chamber music,3436,0,2978.0,458.0,3.680730e-02,2.714286,0.008681,276.0,1.0
6,Frédéric Chopin,3191,0,2312.0,879.0,1.999566e-01,3.325581,0.008418,1096.0,1.0
7,Richard Strauss,3084,1,2648.0,436.0,2.421549e-01,0.904762,0.010023,496.0,2.0
8,Franz Schubert,3015,1,2358.0,657.0,2.794533e-01,1.886364,0.008130,902.0,2.0
9,Instrumental,2815,0,2583.0,232.0,1.517573e-03,0.000000,0.011776,130.0,1.0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [22]:
gc.rank_similarity()
gc.features_df[["node", "category_matches_with_source", "similarity_rank"]].sort_values("similarity_rank", ascending=False).head(10)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

,node,category_matches_with_source,similarity_rank
229,Sonata cycle,2,1.0
0,Baroque,1,0.5
238,Progressive tonality,1,0.5
237,Symphony No. 9 (Schubert),1,0.5
234,Period (music),1,0.5
41,Ostinato,1,0.5
228,Developing variation,1,0.5
46,Musical form,1,0.5
48,Figured bass,1,0.5
57,Tonality,1,0.5


In [23]:
gc.intro_nodes

['Musical form',
 'Classical music era',
 'Movement (music)',
 'Tonality',
 'Exposition (music)',
 'Musical development',
 'Recapitulation (music)',
 'Introduction (music)',
 'Coda (music)',
 'Sonata',
 'Symphony',
 'Concerto',
 'String quartet',
 'Musical analysis']

In [ ]:
def similarity_rank(row):
        try:
            # similarity is penalized by longer paths
            sim_score = row.category_matches_with_source / row.shortest_path_length_from_source        
            # if a path from the source does not exist, it is given a similarity score of 0
            return 0 if np.isnan(sim_score) else sim_score
        except:
            return 0
    
features_df['similarity_rank'] = features_df.apply(similarity_rank, axis=1)
features_df[['node', "similarity_rank", "category_matches_with_source", "shortest_path_length_from_source"]].sort_values("similarity_rank", ascending=False)

In [ ]:
def average_rank(row):
    return np.mean([
        row.degree_ranked,
#         row.centrality_ranked,
        row.dispersion_ranked,
#         row.page_rank_ranked,
#         row.adjusted_reciprocity_ranked,
    ]) * row.category_matches_with_source_ranked * row.shortest_path_length_from_source_ranked 

features_df["rank_average"] = features_df.apply(average_rank, axis=1)

features_df.sort_values("rank_average", ascending=True)

In [ ]:
# features_df.dispersion = features_df.dispersion.fillna(0.0)
# features_df.shortest_path_length_from_source = features_df.shortest_path_length_from_source.fillna(-1)

# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())

# Intro Node Values

In [ ]:
intro_nodes = gc.intro_nodes
intro_nodes_df = None
for node in intro_nodes:
    if df is None:
        intro_nodes_df = pd.DataFrame(features_df[features_df.node == node])
    else:
        intro_nodes_df = pd.concat([intro_nodes_df, pd.DataFrame(features_df[features_df.node == node])])
    
intro_nodes_df

In [ ]:
sns.pairplot(intro_nodes_df, hue="node")